# Interactive Analysis for FAR Pyrometer Data

## Author
Jason Koglin,
Los Alamos National Laboratory

koglin@lanl.gov

## License
This analysis application is part of the catemis package, 
which is distributed as open-source software under a BSD 3-Clause License

© 2022. Triad National Security, LLC. All rights reserved.  LANL Copyright No. C21111.

This program was produced under U.S. Government contract 89233218CNA000001 for Los Alamos
National Laboratory (LANL), which is operated by Triad National Security, LLC for the U.S.
Department of Energy/National Nuclear Security Administration. All rights in the program are
reserved by Triad National Security, LLC, and the U.S. Department of Energy/National Nuclear
Security Administration. The Government is granted for itself and others acting on its behalf a
nonexclusive, paid-up, irrevocable worldwide license in this material to reproduce, prepare
derivative works, distribute copies to the public, perform publicly and display publicly, and to permit
others to do so.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
   may be used to endorse or promote products derived from this software
   without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" 
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE 
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE 
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE 
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import holoviews as hv
import panel as pn
import panel.widgets as pnw
import param

pn.extension('tabulator')
hv.extension('bokeh')

import hvplot.pandas
import hvplot.xarray

from bokeh.models import PrintfTickFormatter
from bokeh.models import HoverTool

from matplotlib.figure import Figure
def null_figure():
    return Figure(figsize=(8,6))

fontsize = {
    'title': 20,
    'xlabel': 18,
    'yticks': 12,
    'ylabel': 18,
    'xticks': 12,
    'zlabel': 18,
    'cticks': 12,
}

In [ ]:
from pathlib import Path
import sys
import os
username = os.getlogin()
default_downloads_path = Path.home()/'Downloads'

In [ ]:
import platform
from pathlib import Path
import catemis

try:
    version = catemis.__version__

except:
    version = 'dev'
    
package = 'catemis'
package_git_url = 'https://github.com/lanl/'+package

# needs update for documentation URL
docs_url = package_git_url

app_site = '{:} v{:}'.format(package, version)

package_path = Path(catemis.__file__).parent
lanl_logo_path = package_path/'template'/'LANL Logo White.png'
jdiv_logo_path = package_path/'template'/'j-div-logo-blue1.png'
if lanl_logo_path.is_file():
    lanl_logo_pn = pn.pane.PNG(str(lanl_logo_path), width=300)
else:
    lanl_logo_pn = None

if jdiv_logo_path.is_file():
    jdiv_logo_pn = pn.Row(
        pn.Spacer(width=50),
        pn.pane.PNG(str(jdiv_logo_path), width=200),
    )

else:
    jdiv_logo_pn = None

app_meta_pn = pn.Column(
    pn.pane.Markdown('###[App Documentation]({:})'.format(docs_url), 
                     align='center', width=320),
    pn.pane.Markdown('###Created by koglin@lanl.gov'),
    pn.pane.Markdown('###[{:}]({:}) v{:}'.format(package, package_git_url, version), 
                     align='center', width=320),
    align='center',
    width=320,
)

In [ ]:
# Default daystr for WESJ paper data
daystr = '20200701'

In [ ]:
from catemis import far_pyro, cathode
default_path = Path(catemis.__file__).parent
log_attrs = ['Sequence', 'Date', 'Time','Temp','Average','Tol','Signal','BB']
hover_attrs = ['Sequence','planck','planck_temp','Temp','intensity','background','eraw']
sum_attrs = ['Sequence', 'log_num', 'Date', 'Time',
             'planck_temp','planck_etemp','planck_norm','planck_enorm','twoband_temp',
             'Temp','Average','Tol','Signal','BB', 
             'ramp_state', 't_exposure',
             ]
sum_attrs0 = ['Sequence', 
              'planck_temp','planck_etemp','twoband_temp',
              'Temp','Tol','Signal','BB', 
              'ramp_state',
              ]
temp_attrs = ['Temp','twoband_temp','planck_temp']

from bokeh.models.widgets.tables import NumberFormatter, BooleanFormatter

formatters = {
    'hours': NumberFormatter(format='0.00'),
    'Tol': NumberFormatter(format='0.0'),
    'Temp': NumberFormatter(format='0.0'),
    'Signal': NumberFormatter(format='0.000'),
    'Average': NumberFormatter(format='0.0'),
    'twoband_temp': NumberFormatter(format='0.0'),
    'planck_temp': NumberFormatter(format='0.0'),
    'planck_etemp': NumberFormatter(format='0.0'),
    'spec_lh': NumberFormatter(format='0.00'),
    'wavelength': NumberFormatter(format='0.0'),
    'crx': NumberFormatter(format='0.0'),
    'emissivity': NumberFormatter(format='0.000'),
    'emissivity0': NumberFormatter(format='0.000'),
    'spec_lh': NumberFormatter(format='0.000'),
    'wave_fit': BooleanFormatter(),
}

from bokeh.models.widgets.tables import CheckboxEditor, NumberEditor, SelectEditor, DateEditor, TimeEditor

bokeh_editors = {
    'wave_fit': CheckboxEditor(),
}

In [ ]:
status_pane = pn.pane.Markdown(object='', width=320, height=60)

In [ ]:
class FAR_Viewer(param.Parameterized):
    """
    Interactive Cathode Image Viewer 
    """    
    load_log = param.Action(lambda x: x.param.trigger('load_log'), 
            label='Load Log')
    load_seq = param.Action(lambda x: x.param.trigger('load_seq'), 
            label='Load Sequences')
    update_crx = param.Action(lambda x: x.param.trigger('update_crx'), 
            label='Update CRX Calibration')
    update_emiss = param.Action(lambda x: x.param.trigger('update_emiss'), 
            label='Update Emissivity')
    update_twoband = param.Action(lambda x: x.param.trigger('update_twoband'), 
            label='Calculate Two-band')
    calculate_planck = param.Action(lambda x: x.param.trigger('calculate_planck'), 
            label='Calculate Planck')
    update_sum = param.Action(lambda x: x.param.trigger('update_sum'), 
            label='Update Summary')
    update_wcut = param.Action(lambda x: x.param.trigger('update_wcut'), 
            label='Update Waveform Mask')

    save_dataset = param.Action(lambda x: x.param.trigger('save_dataset'), 
            label='Save Dataset')
    dataset_file = param.String(label='Save File')

    # Errorbar plots can cause issues with some combination of bokeh, xarray, and hvplot
    # Make False as default, but allow for errorbars to optionally be added.
    plot_errorbars = param.Boolean(default=False)
    
    twoband_pars = param.String(label='Two-band poly1d Params (comma separated)')
    rel_twoband = param.Boolean(default=False, label='Subtract Model')
    twoband_order = param.Integer(default=0, label='Two-band Poly Order')
    twoband_min = param.Number(default=0.1, label='Low/High Min')
    twoband_max = param.Number(default=3, label='Low/High max')
    low_slice_min = param.Integer(label='Low Min [nm]')
    low_slice_max = param.Integer(label='Low Max [nm]')
    high_slice_min = param.Integer(label='High Min [nm]')
    high_slice_max = param.Integer(label='High Max [nm]')
    df_par_widget = pn.widgets.Tabulator(height=150, width=300, 
                                         text_align='right')

    show_residuals = param.Boolean(default=True)
    cols = param.Integer(default=1, label='Columns')
    height = param.Integer(default=500, step=100, label='Plot Height') 
    width = param.Integer(default=800, step=100, label='Plot Width')
    hover_cols = param.ListSelector(
        default=hover_attrs[0:3], 
        objects=hover_attrs,
        label='Hover Params',
    )
    sum_cols = param.ListSelector(
        default=sum_attrs0, 
        objects=sum_attrs,
        label='Spec Params',
    )
    temp_sel = param.ListSelector(
        default=['Temp'],
        objects=['Temp'],
        label='Temperatures',
    )
    xdim = param.Selector(default='hours', 
                          objects=['hours', 'Sequence', 'Temp'],
                          label='X Axis')
    rel_temp = param.Selector(default='None', 
                              objects=['None', 'Temp'],
                              label='Subtract Temperature'
                             )

    path = param.String(default=str(default_path/'FAR'), label='Base Path')
    pyro = param.String(default='FMP2', label='Pyrometer/Folder')
    daystr = param.String(default='20200701', label='Date [yyyymmdd]')
    
    df_log_widget = pn.widgets.Tabulator(height=300, width=700, 
                                         formatters=formatters,
                                         text_align='right',
                                        )
    df_sum_widget = pn.widgets.Tabulator(height=300, width=1000, 
                                         formatters=formatters,
                                         text_align='right',
                                        )
    
    wave_min = param.Integer(default=400, label='Wavelength Min [nm]')
    wave_max = param.Integer(default=2000, label='Wavelength Max [nm]')
        
    emissivity_path = param.String(default=str(default_path/'data'), label='Emissivity Path')
    emissivity_file = param.String(default='emissivity.csv', label='Emissivity File')
    aparam = param.Number(default=0.35, label='Emissivity a param')
    bparam = param.Number(default=0.24, label='Emissivity b param')
    df_emiss_widget = pn.widgets.Tabulator(height=500, width=350, 
                                           formatters=formatters,
                                           text_align='right')
    
    crx_path = param.String(default=str(default_path/'data'), label='FAR CRX Path')
    crx_file = param.String(label='FAR CRX File')
    df_crx_widget = pn.widgets.Tabulator(height=500, width=350, 
                                         formatters=formatters,
                                         text_align='right')

    wcut_path = param.String(default=str(default_path/'data'), label='Waveform Mask Path')
    wcut_file = param.String(label='Waveform Mask File')
    df_wcut_widget = pn.widgets.Tabulator(height=500, width=220, 
                                          formatters=formatters,
                                          editors=bokeh_editors,
                                          text_align='right')

    @param.depends('pyro')
    def reset_all(self):
        """
        Clear all data
        """
        self.ds = None
        self.df_sum_widget.value = None
        self.df_log_widget.value = None
        self.df_crx_widget.value = None
        self.df_wcut_widget.value = None
        self.dlog = None
    
    @param.depends('save_dataset', watch=True)
    def save_to_netcdf(self):
        from pathlib import Path
        if hasattr(self, 'ds'):
            save_file = Path(self.dataset_file)
            if save_file.parent.is_dir():
                status_pane.object = '... saving dataset to {:}'.format(str(save_file))
                self.ds.to_netcdf(str(save_file), engine='h5netcdf')
                
            else:
                status_pane.object = '... path for saveing file not available.'
                status_pane.object += ' -- Create folder and try again'
                
    
    @param.depends('update_crx', watch=True)
    def load_crx_file(self):
        """
        Load calibration CRX file
        """
        from pathlib import Path

        file = Path(self.crx_path)/self.crx_file
        if not file.parent.is_dir():
            status_pane.object = 'ERROR: No path {:}'.format(str(file.parent))
            
        elif not file.is_file():
            status_pane.object = 'ERROR: No file {:}'.format(str(file))
            
        else:
            try:
                status_pane.object = '... loaded {:}'.format(str(file))
                dcrx = far_pyro.load_far_crx(file).to_dataframe()
                self.df_crx_widget.value = dcrx
                status_pane.object = ''
                
            except:
                status_pane.object = 'ERROR: Failed loading {:}'.format(str(file))
                
    def plot_crx(self):
        """
        Plot crx calibration
        """
        dcrx = self.df_crx_widget.value
        if dcrx is not None:
            status_pane.object = '... updating crx plot'
            dcalib = self.get_dcalib().to_xarray()
            plt_kwargs = {
                'fontsize': 16,
                'height': self.height,
                'width': self.width,
                'xlabel': 'Wavelength [nm]',
                'ylabel': 'Calibration (1/crx)',
                'xlim': (self.wave_min, self.wave_max),
                'ylim': (0, dcalib.values.max()*1.1),
            }
            pn_plot = dcalib.hvplot('wavelength', c='r', **plt_kwargs)
            dwcut = self.df_wcut_widget.value
            if dwcut is not None:
                try:
                    dacut = dwcut.to_xarray().interp(wavelength=dcalib.wavelength, method='nearest')
                    pn_plot *= dcalib.where(dacut).hvplot.scatter(
                        'wavelength', 
                        c='m', marker='+', label='Masked', 
                        **plt_kwargs
                    )
                    
                    status_pane.object = ''

                except:
                    status_pane.object = 'Failed adding Mask'
                    
            return pn_plot
                    
    @param.depends('update_wcut', watch=True)
    def load_wcut_file(self):
        """
        Load Waveform Mask File
        """
        from pathlib import Path
        file = Path(self.wcut_path)/self.wcut_file
        if not file.parent.is_dir():
            status_pane.object = 'ERROR: No path {:}'.format(str(file.parent))
            
        elif not file.is_file():
            status_pane.object = 'ERROR: No file {:}'.format(str(file))
            
        else:
            try:
                status_pane.object = '... loading {:}'.format(str(file))
                df = pd.read_csv(file, index_col=0)
                self.df_wcut_widget.value = df
                status_pane.object = ''
                
            except:
                status_pane.object = 'ERROR: Failed Loading {:}'.format(str(file))

        
    @param.depends('update_emiss', watch=True)
    def load_emissivity_file(self):
        """
        Load Emissivity File
        """
        from pathlib import Path
        file = Path(self.emissivity_path)/self.emissivity_file
        if not file.parent.is_dir():
            status_pane.object = 'ERROR: No path {:}'.format(str(file.parent))
            
        elif not file.is_file():
            status_pane.object = 'ERROR: No file {:}'.format(str(file))
            
        else:
            try:
                status_pane.object = '... loading {:}'.format(str(file))
                dsb = cathode.load_emissivity(
                    str(file), 
                    aparam=self.aparam, 
                    bparam=self.bparam,
                )
                dfb = dsb.reset_coords()[['emissivity','emissivity0']].to_dataframe()
                self.df_emiss_widget.value = dfb
                status_pane.object = ''
                
            except:
                status_pane.object = 'ERROR: Failed Loading {:}'.format(str(file))
            
    def plot_emiss(self):
        """
        Plot Emissivity
        """
        df = self.df_emiss_widget.value
        plt_kwargs = {
            'fontsize': 16,
            'height': self.height,
            'width': self.width,
            'xlabel': 'Wavelength [nm]',
            'ylabel': 'Emissivity',
            'xlim': (self.wave_min, self.wave_max),
        }
        if df is not None:
            status_pane.object = '... plotting emissivity'
            df = self.df_emiss_widget.value
            pn_plot = df.hvplot('wavelength', **plt_kwargs)
            status_pane.object = ''
            return pn_plot

    def set_pyro_defaults(self):
        """
        Set defaults for FAR FMP1/FMP2 pyrometers
        """
        pyro = self.pyro
        if pyro == 'FMP1':
            wave_range = (900,1700)
            twoband_pars = [ 108.1, -245.2,  987.3,  256.1]
            low_slice = (1275,1300)
            high_slice = (1610,1630)
            lh_range = (0.3, 1.1)

        else:
            wave_range = (500,1000)
            twoband_pars = [367.8, 321.0]
            low_slice = (750,800)
            high_slice = (880,900)
            lh_range = (0.5, 2.5)
            if pyro != 'FMP2':
                status_pane.object = '... Setting defaults for {:} with FMP2'.format(pyro)

        self.crx_file = self.pyro+'.crx'
        self.wcut_file = self.pyro+'_mask.csv'
        self.wave_min = wave_range[0]
        self.wave_max = wave_range[1]
        self.low_slice_min = low_slice[0]
        self.low_slice_max = low_slice[1]
        self.high_slice_min = high_slice[0]
        self.high_slice_max = high_slice[1]
        self.twoband_min = lh_range[0]
        self.twoband_max = lh_range[1]
        self.twoband_pars = ','.join([str(p) for p in twoband_pars])
        # Set order of two-band from defaults
        self.twoband_order = len(twoband_pars)-1
        self.dataset_file = str(default_downloads_path/'{:}_{:}.nc'.format(pyro, self.daystr))
    
    @param.depends('load_log', watch=True)
    def load_log_file(self):
        from pathlib import Path
        status_pane.object = '... Loading Log Files'
        self.reset_all()

        try:
            dlog = far_pyro.load_pyro(pyro=self.pyro, 
                    daystr=self.daystr, 
                    pyro_path=Path(self.path),
                    load_waveforms=False)
        except:
            status_pane.object = '... Failed Loading Log from {:}'.format(self.path)

        try:
            df = dlog.reset_coords()[[a for a in log_attrs if a in dlog]].to_dataframe()
            self.df_log_widget.value = df
            self.dlog = dlog

        except:
            status_pane.object = '... Failed Updating Log Table'

        self.set_pyro_defaults()
        status_pane.object = 'Select Sequences in Log and Load'
    
    def get_twoband_pars(self):
        if self.twoband_pars:
            try:
                twoband_pars = [float(p) for p in self.twoband_pars.split(',')]
                
            except:
                twoband_pars = False
            
        else:
            twoband_pars = None
            
        return twoband_pars
          
    @param.depends('load_seq', watch=True)
    def load_spectra(self):
        from pathlib import Path
        if not hasattr(self, 'dlog'):
            self.param.trigger('load_log')
        
        self.df_sum_widget.value
        self.rel_temp = 'None'
        self.temp_sel = []
        self.param.rel_temp.objects = ['None', 'Temp']
        if self.xdim == 'planck_temp':
            self.xdim = 'hours'
        self.param.xdim.objects = ['hours', 'Sequence', 'Temp']
        self.param.temp_sel.objects = ['Temp']

        status_pane.object = '... Loading Spectra'
        seq = list(self.df_log_widget.selected_dataframe.index.values)
        ds = far_pyro.load_pyro(pyro=self.pyro, 
            daystr=self.daystr, 
            pyro_path=Path(self.path),
            sequence=seq,
            status=status_pane.object)

        coords = [c for c in ['raw_intensity', 'background', 'intensity'] if c in ds]
        self.ds = ds.set_coords(coords)
        self.update_sum_df()
        self.temp_sel = self.param.temp_sel.objects
        status_pane.object = 'Calculate Planck or Select Other Sequences'
        
    @param.depends('update_twoband', watch=True)
    def update_twoband_calcs(self):
        """
        Update two-band temperature calculations
        """
        if not hasattr(self, 'ds'):
            status_pane.object = 'Load Sequences required before update two-band'
            return

        twoband_pars = self.get_twoband_pars()  
        estr = None
        if twoband_pars is False:
            estr = 'Error Interpretting two-band pars: {:}'.format(twoband_pars)
            status_pane.object = estr
            return
        
        status_pane.object = '... updating two-band temperature calculations'
        low_slice = slice(self.low_slice_min, self.low_slice_max)
        high_slice = slice(self.high_slice_min, self.high_slice_max)
        try:
            self.ds = far_pyro.pyro_temperature_twoband(
                self.ds,
                twoband_pars=twoband_pars,
                low_slice=low_slice,
                high_slice=high_slice,
            )
            
            self.update_sum_df()
            attr = 'twoband_temp'
            if attr not in self.param.rel_temp.objects:
                self.param.rel_temp.objects = list(self.param.rel_temp.objects) + [attr]

            if attr not in self.param.xdim.objects:
                self.param.xdim.objects = list(self.param.xdim.objects) + [attr]

            if attr not in self.param.temp_sel.objects:
                self.param.temp_sel.objects = list(self.param.temp_sel.objects) + [attr]

            status_pane.object = 'Two-band Temperature Calculated'
            self.temp_sel = self.param.temp_sel.objects

        except:
            status_pane.object = 'Failed updating two-band calculations'
          
    def get_wave_fit(self):
        dwcut = self.df_wcut_widget.value
        if dwcut is None:
            status_pane.object = '... Loading default crx calibration'
            self.param.trigger('update_wcut')
            dwcut = self.df_wcut_widget.value

        if dwcut is not None:
            try:
                dwcut = self.df_wcut_widget.value
                dacut = dwcut.to_xarray()
                wave_fit = dacut.wave_fit.sel(wavelength=self.ds.wavelength, method='nearest')
                return wave_fit
                
            except:
                status_pane.object = 'Error adding waveform mask'
        
    def get_dcalib(self):
        dcrx = self.df_crx_widget.value
        if dcrx is None:
            status_pane.object = '... Loading default crx calibration'
            self.param.trigger('update_crx')
            dcrx = self.df_crx_widget.value
            
        dcalib = dcrx.dcalib
        return dcalib

    def get_emissivity(self):
        dfe = self.df_emiss_widget.value
        if dfe is None:
            status_pane.object = '... Loading default emissivity'
            self.param.trigger('update_emiss')
            dfe = self.df_emiss_widget.value

        emissivity = dfe.emissivity.to_xarray()
        return emissivity
    
    @param.depends('calculate_planck', watch=True)
    def add_planck(self):
        """
        Add Planck Temperature Fits
        """
        if not hasattr(self, 'ds'):
            status_pane.object = 'Load Sequences required before Calculate Planck'
            return

        dcalib = self.get_dcalib()
        wave_fit = self.get_wave_fit()
        emissivity = self.get_emissivity()
        
        status_pane.object = '... Calculating Planck Temperature'

        self.ds = far_pyro.far_temperature(
                self.ds, 
                dcalib=dcalib,
                emissivity=emissivity,
                wave_fit=wave_fit,
                status=status_pane.object,
        )  
        self.update_sum_df()
        attr = 'planck_temp'
        if attr not in self.param.rel_temp.objects:
            self.param.rel_temp.objects = list(self.param.rel_temp.objects) + [attr]

        if attr not in self.param.xdim.objects:
            self.param.xdim.objects = list(self.param.xdim.objects) + [attr]

        if attr not in self.param.temp_sel.objects:
            self.param.temp_sel.objects = list(self.param.temp_sel.objects) + [attr]

        self.temp_sel = self.param.temp_sel.objects
        status_pane.object = 'Planck Calculated'
    
    @param.depends('update_sum', 'sum_cols', watch=True)
    def update_sum_df(self):
        if hasattr(self, 'ds'):
            status_pane.object = '... Updating Summary'
            sattrs = ['hours'] + [a for a in self.sum_cols if a in self.ds]
            if 'spec_lh' in self.ds:
                sattrs.append('spec_lh')
                
            df = self.ds.reset_coords()[sattrs].to_dataframe()
            self.df_sum_widget.value = df
            status_pane.object = ''
    
    def plot_spectra(self):
        """
        Plot spectra
        """
        ds = getattr(self, 'ds', None)

        if ds is None:
            return null_figure()

        if 'raw' in ds:
            plt_kwargs = {
                'fontsize': 16,
                'height': self.height,
                'width': self.width,
                'xlabel': 'Wavelength [nm]',
                'xlim': (self.wave_min, self.wave_max),
                'hover_cols': self.hover_cols,
            }
            
            pn_plot =  ds.raw.hvplot(
                x='wavelength', label='Measured', 
                ylabel=' [ADU]', c='r', **plt_kwargs
            )

            add_legend_position = False
            if self.plot_errorbars:
                add_legend_position = True
                pn_plot *= ds.raw.hvplot.errorbars(
                    'wavelength', 'raw', 
                    yerr1='eraw', yerr2='eraw', 
                    c='r', upper_head=None, lower_head=None,
                    **plt_kwargs
                )
                
            if 'planck' in ds:
                add_legend_position = True
                pn_plot *= ds.planck.hvplot(
                    x='wavelength', 
                    c='k', label='Planck Fit',
                    **plt_kwargs
                )
                
            if add_legend_position:
                pn_plot = pn_plot.opts(legend_position='top_left')
            
            app_plot = pn_plot 

            if 'planck' in ds and self.show_residuals:
                das = (((ds.raw+ds.background)/(ds.planck+ds.background)-1)*100).rename('res')
                das.coords['eres'] = ds.eraw/(ds.planck+ds.background)*100
                pn_plot = das.hvplot(
                    x='wavelength', ylabel='Residuals [%]', 
                    ylim=[-4,6], c='r', **plt_kwargs
                )
                if self.plot_errorbars:
                    pn_plot *= das.hvplot.errorbars(
                        'wavelength', y='res', yerr1='eres', yerr2='eres', 
                        c='r', upper_head=None, lower_head=None, 
                        **plt_kwargs
                    )
                
                app_plot += pn_plot
                    
                if self.cols == 1:
                    app_plot = app_plot.cols(self.cols)
        
            return app_plot

    def plot_temps(self):
        """
        Plot temperatures
        """
        xlabel_dict = {
            'hours': 'Time [h]',
            'Temp': 'FAR Temperature [C]',
            'twoband_temp': 'Two-band Temperature [C]',
            'planck_temp': 'Planck Temperature [C]',
        }
        xsub_dict = {
            'Temp': 'FAR',
            'twoband_temp': 'Two-band',
            'planck_temp': 'Planck',
        }

        dlog = getattr(self, 'dlog', None)
        if dlog is None:
            return null_figure()

        pn_plot = None
        df0 = self.df_sum_widget.value
        if df0 is None:
            return null_figure()

        status_pane.object = 'Updating Temperature Plot'
        df = df0.copy()
        df['xdim'] = df[self.xdim]
        xdim = 'xdim'
        ylabel = 'Temperature [C]'
        if self.rel_temp in df:
            ylabel = 'Relative Temperature [C]'
            for attr in self.temp_sel:
                if attr in df and attr != self.rel_temp:
                    df[attr] -= df[self.rel_temp]

        plt_kwargs = {
            'fontsize': 16,
            'height': self.height,
            'width': self.width,
            'hover_cols': temp_attrs,
            'ylabel': ylabel,
            'xlabel': xlabel_dict.get(self.xdim, self.xdim),
        }

        pn_plot = None
        attr = 'Temp'
        eattr = 'Tol'
        label='FAR'
        if self.rel_temp in df:
            label += ' - {:}'.format(xsub_dict.get(self.rel_temp, self.rel_temp))

        if attr in self.temp_sel and attr not in self.rel_temp:
            pnp = df.hvplot.scatter(
                x=xdim, y=attr, label=label, 
                c='r', marker='o', 
                **plt_kwargs
            )
            
            if self.plot_errorbars:
                pnp *= df.hvplot.errorbars(
                    x=xdim, y=attr, yerr1=eattr, yerr2=eattr, 
                    c='r', upper_head=None, lower_head=None, 
                    **plt_kwargs
                )
                
            if pn_plot is None:
                pn_plot = pnp

            else:
                pn_plot *= pnp

        attr = 'planck_temp' 
        eattr = 'planck_etemp' 
        label='Planck'
        if self.rel_temp in df:
            label += ' - {:}'.format(xsub_dict.get(self.rel_temp, self.rel_temp))
            
        if attr in self.temp_sel and attr in df and attr not in self.rel_temp:
            pnp = df.hvplot.scatter(
                x=xdim, y='planck_temp', label=label, 
                c='b', marker='*', 
                **plt_kwargs
            )
            if self.plot_errorbars:
                pnp *= df.hvplot.errorbars(
                    x=xdim, y='planck_temp', 
                    yerr1='planck_etemp', yerr2='planck_etemp', 
                    c='b', upper_head=None, lower_head=None, 
                    **plt_kwargs
                )
            
            if pn_plot is None:
                pn_plot = pnp
            
            else:
                pn_plot *= pnp

        attr = 'twoband_temp'
        label='Two-band'
        if self.rel_temp in df:
            label += ' - {:}'.format(xsub_dict.get(self.rel_temp, self.rel_temp))

        if attr in self.temp_sel and attr not in self.rel_temp:
            pnp = df.hvplot.scatter(
                x=xdim, y=attr, label=label, 
                c='g', marker='+', 
                **plt_kwargs
            )
            
            if pn_plot is None:
                pn_plot = pnp
            
            else:
                pn_plot *= pnp
                
        status_pane.object = ''
        return pn_plot        
    
    def plot_two_band(self):
        """
        Plot temperature vs two-band spectrum ratio
        """        
        pn_plot = None
        df0 = self.df_sum_widget.value
        if df0 is None or 'spec_lh' not in df0:
            return null_figure()

        if not hasattr(self, 'ds') or self.ds is None:
            return null_figure()

        status_pane.object = 'Updating Two-band Plot'
        # Calculate two-band model
        ds = self.ds
        if 'planck_temp' in ds:
            attr = 'planck_temp'
        else:
            attr = 'Temp'

        df = df0.copy()

        twoband_pars = self.get_twoband_pars()
        dict_pars = {}
        dict_pars['temp_lh'] = twoband_pars
        dpar = xr.DataArray(np.zeros((len(self.temp_sel), self.twoband_order+1)), 
                            dims=('fit_name','pars'), name='pars_fit'
                           )
        
        for ifit, attr in enumerate(self.temp_sel):
            pfit = np.polyfit(df.spec_lh, df[attr], self.twoband_order)
            dict_pars[attr+'_fit'] = pfit
            dpar.loc[ifit] = pfit

        dpar['fit_name'] = self.temp_sel
        
        vlh = np.arange(self.twoband_min,self.twoband_max,0.01)
        ada = []
        par_formatters = {}
        for attr, plist in dict_pars.items():
            func1d = np.poly1d(plist)
            vtemps = func1d(vlh)
            da = xr.DataArray(vtemps, dims=('spec_lh'), coords={'spec_lh': vlh}, name=attr)
            ada.append(da)
            df[attr] = da.interp(spec_lh=df.spec_lh)
        
        da = xr.merge(ada)
        
        par_formatters = {}
        for i in range(self.twoband_order+1):
            par_formatters[i] = NumberFormatter(format='0.00')

        self.df_par_widget.value = dpar.to_dataframe().unstack().pars_fit
        self.df_par_widget.formatters = par_formatters

        xdim = 'spec_lh'
        plt_kwargs = {
            'fontsize': 16,
            'height': self.height,
            'width': self.width,
            'hover_cols': temp_attrs,
            'ylabel': 'Temperature [C]',
            'xlabel': 'Two-band Low/High Ratio',
        }

        rel_temp = 'temp_lh'
        ylabel = 'Temperature [C]'
        if self.rel_twoband:
            da -= da['temp_lh']
            
            ylabel = 'Relative Temperature [C]'
            for attr in self.temp_sel:
                if attr in df and attr != rel_temp:
                    df[attr] -= df[rel_temp]

            pn_plot = da['temp_lh'].hvplot(
                x='spec_lh', c='k', 
                **plt_kwargs
            )

        else:
            pn_plot = da['temp_lh'].hvplot(
                x='spec_lh', 
                label='Two-band Model', c='g', 
                **plt_kwargs
            )
        
        attr = 'Temp'
        eattr = 'Tol'
        label='FAR'
        flabel=label+' poly fit'
        if self.rel_twoband:
            label += ' - Two-band Model'

        if attr in self.temp_sel:
            pnp = df.hvplot.scatter(
                x=xdim, y=attr, label=label, 
                c='r', marker='o', **plt_kwargs
            )
            
            if self.plot_errorbars:
                pnp *= df.hvplot.errorbars(
                    x=xdim, y=attr, yerr1=eattr, yerr2=eattr, 
                    c='r', upper_head=None, lower_head=None, 
                    **plt_kwargs
                )

            pnp *= da[attr+'_fit'].hvplot(
                x='spec_lh', c='r', 
                label=flabel, 
                **plt_kwargs
            )

            if pn_plot is None:
                pn_plot = pnp

            else:
                pn_plot *= pnp

        attr = 'planck_temp' 
        eattr = 'planck_etemp' 
        label='Planck'
        flabel=label+' poly fit'
        if self.rel_twoband:
            label += ' - Two-band Model'
            
        if attr in self.temp_sel and attr in df:
            pnp = df.hvplot.scatter(x=xdim, y='planck_temp', label=label, 
                                    c='b', marker='*', **plt_kwargs)
            if self.plot_errorbars:
                pnp *= df.hvplot.errorbars(
                    x=xdim, y='planck_temp', 
                    yerr1='planck_etemp', yerr2='planck_etemp', 
                    c='b', upper_head=None, lower_head=None, 
                    **plt_kwargs
                )

            pnp *= da[attr+'_fit'].hvplot(
                x='spec_lh', c='b', 
                label=flabel, 
                **plt_kwargs
            )

            if pn_plot is None:
                pn_plot = pnp

            else:
                pn_plot *= pnp

        attr = 'twoband_temp'
        label='Two-band'
        if self.rel_twoband:
            label += ' - Two-band Model'
            
        if attr in self.temp_sel:
            pnp = df.hvplot.scatter(
                x=xdim, y=attr, label=label, 
                c='g', marker='+', 
                **plt_kwargs
            )
            
            if pn_plot is None:
                pn_plot = pnp
                
            else:
                pn_plot *= pnp

        status_pane.object = ''
        return pn_plot        
    
    def cam_params(self):
        pn_params = pn.Column(
            pn.Row(
                self.param.load_log,
                width=320,
            ),
            self.param.path,
            pn.Row(
                self.param.daystr,
                self.param.pyro,
                width=320,
            ),
            pn.Row(
                self.param.load_seq,
                width=320,
            ),
            pn.Row(
                status_pane,
                height=50,
                width=320,
            ),
            pn.Row(
                self.param.update_twoband,
                self.param.calculate_planck,
                width=320,
            ),
            pn.Row(
                self.param.wave_min,
                self.param.wave_max,
                width=320,
            ),
            pn.Row(
                self.param.width, 
                self.param.height, 
                width=320,
            ),
            pn.Row(
                pn.layout.Divider(),
                width=320,
            ),
            pn.Column(
                self.param.save_dataset,
                self.param.dataset_file,
                width=320,
            ),
        )

        return pn_params
    
    def app(self):
        pn_app = pn.template.MaterialTemplate(
            title='FAR Pyrometer Viewer', 
            logo=str(lanl_logo_path),
            header_background='#000F7E',
            theme=pn.template.DefaultTheme,
        )

        pn_app.sidebar.append(jdiv_logo_pn)
        pn_app.sidebar.append(self.cam_params)
        pn_app.sidebar.append(pn.layout.Divider())
        pn_app.sidebar.append(app_meta_pn)

        pn_app.main.append(
            pn.Column(
                pn.Card(
                    pn.Column(
                        pn.Row(
                            self.df_log_widget.param.width, 
                            self.df_log_widget.param.height, 
                            width=200,
                        ),
                        self.df_log_widget,
                    ),
                    title='FAR Log'
                ),
                pn.Card(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                self.param.xdim,
                                width=150,
                            ),
                            pn.Column(
                                self.param.rel_temp,
                                width=150,
                            ),
                            pn.Column(
                                self.param.temp_sel,
                                width=150,
                            ),
                        ),
                        self.plot_temps,
                    ),
                    title='Temperature Plot'
                ),
                pn.Card(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                pn.Row(
                                    self.df_sum_widget.param.width, 
                                    self.df_sum_widget.param.height, 
                                    width=200,
                                ),
                                self.param.update_sum,
                                width=200,
                            ),
                            pn.Column(
                                self.param.sum_cols,
                                width=200,
                            ),
                        ),
                        self.df_sum_widget,
                    ),
                    title='Temperature Table'
                ),
                pn.Card(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                self.param.hover_cols,
                                width=200,
                            ),
                            pn.Column(
                                self.param.cols,
                                self.param.show_residuals,
                                width=100,
                            ),
                        ),
                        self.plot_spectra,
                    ),
                    title='FAR Spectra',
                ),
                pn.Card(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                self.param.update_crx,
                                width=200,
                            ),
                            pn.Column(
                                self.param.crx_path,
                                width=350,
                            ),
                            pn.Column(
                                self.param.crx_file,
                                width=150,
                            ),
                        ),
                        pn.Row(
                            pn.Column(
                                self.param.update_wcut,
                                width=200,
                            ),
                            pn.Column(
                                self.param.wcut_path,
                                width=350,
                            ),
                            pn.Column(
                                self.param.wcut_file,
                                width=150,
                            ),
                        ),
                        pn.Row(
                            self.df_crx_widget,
                            self.df_wcut_widget,
                            self.plot_crx,
                        ),
                    ),
                    title='FAR Calibration',
                ),
                pn.Card(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                self.param.update_emiss,
                                width=200,
                            ),
                            pn.Column(
                                self.param.emissivity_path,
                                self.param.emissivity_file,
                                width=350,
                            ),
                            pn.Column(
                                self.param.aparam,
                                self.param.bparam,
                                width=180,
                            ),
                        ),
                        pn.Row(
                            self.df_emiss_widget,
                            self.plot_emiss,
                        ),
                    ),
                    title='Emissivity',
                ),
                pn.Card(
                    pn.Column(
                        pn.Row(
                            pn.Column(
                                self.param.temp_sel,
                                width=150,
                            ),
                            pn.Column(
                                self.param.low_slice_min,
                                self.param.high_slice_min,
                                width=100,
                            ),
                            pn.Column(
                                self.param.low_slice_max,
                                self.param.high_slice_max,
                                width=100,
                            ),
                            pn.Column(
                                self.param.twoband_min,
                                self.param.twoband_max,
                                width=100,
                            ),
                            pn.Column(
                                self.param.twoband_order,
                                self.param.rel_twoband,
                                width=150,
                            ),
                            pn.Column(
                                self.param.twoband_pars,
                                self.param.update_twoband,
                                width=300,
                            ),
                        ),
                        self.plot_two_band,
                        pn.Row(pn.pane.Markdown('#### Poly Fit Params')),
                        self.df_par_widget,
                    ),
                    title='Two-Band Plot'
                ),
            ),
        )

        return pn_app    
    
viewer = FAR_Viewer()

In [ ]:
# To launch straight from Jupyter Notebook uncomment and execute this line
#viewer.app().show()

In [ ]:
# For use with command line as in documentation
# >>> panel serve --show far_viewer.ipynb --port=5052
viewer.app().servable()

In [ ]:
status_pane.object = 'Load Log or Update File Options'